In [1]:
import numpy as np
import pandas as pd

https://en.wikipedia.org/wiki/Decision_tree

# Decision Tree

In [71]:
class DecisionTree():
    """
    
    Decision Tree Classifier
    
    Attributes:
        root: Root Node of the tree.
        max_depth: Max depth allowed for the tree
        size_allowed : Min_size split, smallest size allowed for split 
        n_features: Number of features to use during building the tree.(Random Forest)
        n_split:  Number of split for each feature. (Random Forest)
    
    """

    def __init__(self, max_depth = 1000, size_allowed = 1, n_features = None, n_split = None):
        """
            Initializations for class attributes.
        """
        self.root = None             
        self.max_depth = max_depth         
        self.size_allowed = size_allowed      
        self.n_features = n_features        
        self.n_split = n_split           
    
    
    class Node():
        """
            Node Class for the building the tree.

            Attribute: 
                threshold: The threshold like if x1 < threshold, for spliting.
                feature: The index of feature on this current node.
                left: Pointer to the node on the left.
                right: Pointer to the node on the right.
                pure: Bool, describe if this node is pure.
                predict: Class, indicate what the most common Y on this node.

        """
        def __init__(self, threshold = None, feature = None):
            """
                Initializations for class attributes.
            """
            self.threshold = threshold   
            self.feature = feature    
            self.left = None
            self.right = None
            self.pure = None
            self.depth = None
            self.predict = None
    
    
    def entropy(self, lst):
        """
            Function Calculate the entropy given lst.
            
            Attributes: 
                entro: variable store entropy for each step.
                classes: all possible classes. (without repeating terms)
                counts: counts of each possible classes.
                total_counts: number of instances in this lst.
                
            lst is vector of labels.
        """
        
        entro = 0  
        classes = set(lst)  
        counts = [list(lst).count(c) for c in classes] 
        total_counts = len(lst)    
        for count in counts:       
            if count != 0:    
                p = count/total_counts    
                entro -= p * np.log2(p)
        return entro

    def information_gain(self, lst, values, threshold):
        """
            Function Calculate the information gain, by using entropy function.
            
            lst is vector of labels.
            values is vector of values for individule feature.
            threshold is the split threshold we want to use for calculating the entropy. 
        """
        
        left_prop = sum([1 for val in values if val < threshold]) / len(values) 
        right_prop = 1 - left_prop    

        left_labels = [
            label for i, label in enumerate(lst) if values[i] < threshold
        ]
        right_labels = [
            label for i, label in enumerate(lst) if values[i] >= threshold
        ]

        assert sorted(left_labels + right_labels) == sorted(lst)

        left_entropy = self.entropy(left_labels)    
        right_entropy = self.entropy(right_labels)   
        
        information_gain = (
            self.entropy(lst) 
            - 
            (left_prop * left_entropy + right_prop * right_entropy)
        )

        return information_gain  
    
    def find_rules(self, data):
        
        """
            Helper function to find the split rules.
            
            data is a matrix or 2-D numpy array, represnting training instances. 
            Each training instance is a feature vector. 
        """
        rules = []        
        for col in range(data.shape[1]):          
            unique_values = np.unique(data[:, col])       
            mid_points = [
                (unique_values[i] + unique_values[i+1]) / 2 for i in range(len(unique_values) - 1)
            ]        
            rules.append(mid_points)             
        return rules
    
    def next_split(self, data, label):
        """
            Helper function to find the split with most information gain, by using find_rules, and information gain.
            
            data is a matrix or 2-D numpy array, represnting training instances. 
            Each training instance is a feature vector. 
            
            label contains the corresponding labels. There might be multiple (i.e., > 2) classes.
        """
        
        rules = self.find_rules(data)             
        max_info = -1          
        num_col = None          
        threshold = None       
        entropy_y = None   

        """
            Check Number of features to use, None means all featurs. (Decision Tree always use all feature)
            If n_features is a int, use n_features of features by random choice. 
            If n_features == 'sqrt', use sqrt(Total Number of Features ) by random choice.
        """
        if num_col is None:
            index_col = list(range(data.shape[1]))
        else:
            if num_col == 'sqrt': 
                num_index = int(np.sqrt(data.shape[1]))
            else:
                num_index = num_col
            np.random.seed()  
            index_col = np.random.choice(
                data.shape[1], num_index, replace=False
            )
        
        """
            Do the similar selection we did for features, n_split take in None or int or 'sqrt'.
            For all selected feature and corresponding rules, we check it's information gain.       
        """
        for i in index_col:

            index_rules = []
            num_rules = 0
            
            if self.n_split is None:
                index_rules = rules[i]
            elif len(rules[i]) > 0:
                if self.n_split == 'sqrt':
                    num_rules = int(np.sqrt(len(rules[i])))
                else:
                    num_rules = self.n_split
                np.random.seed()
                index_rules = np.random.choice(
                    rules[i], num_rules, replace=False
                )
            
            for rule in index_rules:
                info = self.information_gain(label, data[:, i], rule)     
                if info > max_info:  
                    max_info = info
                    num_col = i
                    threshold = rule
                    print(f'max_info: {max_info}, num_col: {num_col}, threshold: {threshold}')

        return threshold, num_col
        
    def build_tree(self, X, y, depth):
        
            """
                Helper function for building the tree.
                
                First build the root node.
            """
            
            # self.threshold = threshold   
            # self.feature = feature    
            # self.left = None
            # self.right = None
            # self.pure = None
            # self.depth = None
            # self.predict = None
            
            first_threshold, first_feature = self.next_split(X, y)
            current = self.Node(first_threshold, first_feature)  
            current.depth = depth
            
            
            """
                TODO: 15. Base Case 1: Check if we pass the max_depth, check if the first_feature is None, min split size.
                          If some of those condition met, change current to pure, and set predict to the most popular label
                          and return current
            """
            if depth > self.max_depth or first_feature is None:
                current.predict = max(set(list(y)), key=list(y).count)
                current.pure = True
                return current
            
            """
               Base Case 2: Check if there is only 1 label in this node, change current to pure, and set predict to the label
            """
            
            if len(np.unique(y)) == 1:
                current.predict = y[1]
                current.pure = True
                return current
            
            """
                TODO: 16. Find the left node index with feature i <= threshold  Right with feature i > threshold.
            """
            
            left_index = X[:, first_feature] <= first_threshold
            right_index = X[:, first_feature] > first_threshold
            
            """
                TODO: 17. Base Case 3: If we either side is empty, change current to pure, and set predict to the label
            """
            
            if len(left_index)==0 or len(right_index)==0:
                current.predict = y[first_feature]
                current.pure = True 
                return current
            
            
            left_X, left_y = X[left_index,:], y[left_index]
            current.left = self.build_tree(left_X, left_y, depth + 1)
                
            right_X, right_y = X[right_index,:], y[right_index]
            current.right = self.build_tree(right_X, right_y, depth + 1)
            
            return current
    

        
    def fit(self, X, y):
        
        """
            The fit function fits the Decision Tree model based on the training data. 
            
            X_train is a matrix or 2-D numpy array, represnting training instances. 
            Each training instance is a feature vector. 

            y_train contains the corresponding labels. There might be multiple (i.e., > 2) classes.
        """
        self.root = self.build_tree(X, y, 1)
        
        return self
            
    def ind_predict(self, inp):
        """
            Predict the most likely class label of one test instance based on its feature vector x.
            
            TODO: 18. Modify the following while loop to get the prediction.
                      Stop condition we are at a node is pure.
                      Trace with the threshold and feature.
                19. Change return self.for_running to appropiate value.
        """
        cur = self.root  
        while not cur.pure:  
            
            feature = cur.feature  
            threshold = cur.threshold 
            
            if inp[feature] <= threshold:  
                cur = cur.left
            else:
                cur = cur.right
        return cur.predict
    
    def predict(self, inp):
        """
            X is a matrix or 2-D numpy array, represnting testing instances. 
            Each testing instance is a feature vector. 
            
            Return the predictions of all instances in a list.
            
            TODO: 20. Revise the following for-loop to call ind_predict to get predictions.
        """
        
        result = []
        for i in range(inp.shape[0]):
            result.append(self.ind_predict(inp[i]))
        return result
    




In [72]:
threshold = 0.6
X[X[:, 1] > threshold]

array([[ 7.4  ,  0.7  ,  0.   , ...,  3.51 ,  0.56 ,  9.4  ],
       [ 7.8  ,  0.88 ,  0.   , ...,  3.2  ,  0.68 ,  9.8  ],
       [ 7.8  ,  0.76 ,  0.04 , ...,  3.26 ,  0.65 ,  9.8  ],
       ...,
       [ 5.4  ,  0.74 ,  0.09 , ...,  3.67 ,  0.56 , 11.6  ],
       [ 6.8  ,  0.62 ,  0.08 , ...,  3.42 ,  0.82 ,  9.5  ],
       [ 5.9  ,  0.645,  0.12 , ...,  3.57 ,  0.71 , 10.2  ]])

In [73]:
url_Wine = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
wine = pd.read_csv(url_Wine, delimiter=';')


In [74]:
X = np.array(wine)[:, :-1]
y = np.array(wine)[:, -1]
y = np.array(y.flatten())

In [75]:
from sklearn.model_selection import train_test_split

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state = 0)

In [77]:
clf = DecisionTree()
clf.fit(X_train, y_train)

max_info: 0.0010691910915683334, num_col: 0, threshold: 4.800000000000001
max_info: 0.0017705552953275472, num_col: 0, threshold: 4.95
max_info: 0.005977595369415001, num_col: 0, threshold: 5.05
max_info: 0.008872066308834414, num_col: 0, threshold: 5.15
max_info: 0.010287499567904979, num_col: 0, threshold: 5.25
max_info: 0.011085693467390634, num_col: 0, threshold: 5.35
max_info: 0.013892355320910488, num_col: 0, threshold: 5.45
max_info: 0.01598223443187896, num_col: 0, threshold: 5.55
max_info: 0.017827209886106532, num_col: 0, threshold: 7.85
max_info: 0.018137232696566752, num_col: 0, threshold: 8.05
max_info: 0.020675172021679566, num_col: 0, threshold: 8.149999999999999
max_info: 0.021939671863775656, num_col: 0, threshold: 8.45
max_info: 0.02258012039385382, num_col: 0, threshold: 8.649999999999999
max_info: 0.022881149448965576, num_col: 0, threshold: 10.05
max_info: 0.02335137773185103, num_col: 1, threshold: 0.275
max_info: 0.03516493002247989, num_col: 1, threshold: 0.2850

### Train Accuracy should be 1

In [88]:
pred = clf.predict(X_train)
(pred == y_train).mean()

1.0

### Test Accuracy should be around 0.62

In [89]:
pred = clf.predict(X_test)

In [90]:
(pred == y_test).mean()

0.628125

https://en.wikipedia.org/wiki/Random_forest

In [15]:
class RandomForest():
    
    """
    
    RandomForest Classifier
    
    Attributes:
        n_trees: Number of trees. 
        trees: List store each individule tree
        n_features: Number of features to use during building each individule tree.
        n_split: Number of split for each feature.
        max_depth: Max depth allowed for the tree
        size_allowed : Min_size split, smallest size allowed for split 
    
    """
    
    def __init__(self,n_trees = 10, n_features = 'sqrt', n_split = 'sqrt', max_depth = None, size_allowed = 1):
        
        """
            Initilize all Attributes.
            
            TODO: 1. Intialize n_trees, n_features, n_split, max_depth, size_allowed.
        """
        self.n_trees = n_trees
        self.trees = []
        self.n_features = n_features
        self.n_split = n_split
        self.max_depth = max_depth
        self.size_allowed = size_allowed
        
        
    def fit(self, X, y):
        """
        The fit function fits the Random Forest model based on the training data. 
        
        X_train is a matrix or 2-D numpy array, representing training instances. 
        Each training instance is a feature vector. 
        
        y_train contains the corresponding labels. There might be multiple (i.e., > 2) classes.
        
        TODO: 2. Modify the following for loop to create n_trees trees by calling DecisionTree we created.
                 Pass in all the attributes.
        """
        self.for_running = y[4]
        
        for i in range(self.n_trees):
            np.random.seed()
            temp_clf = DecisionTree(n_features=self.n_features, n_split=self.n_split, max_depth=self.max_depth, size_allowed=self.size_allowed)
            temp_clf.fit(X, y)
            self.trees.append(temp_clf)
        return self
            
    def ind_predict(self, inp):
        
        """
            Predict the most likely class label of one test instance based on its feature vector x.
        
            TODO: 4. Modify the following code to predict using each Decision Tree.
        """
        result = []
        for tree in self.trees:
            result.append(tree.predict(inp))
            
                
        """
            TODO: 5. Modify the following code to find the correct prediction using majority rule.
                     If there is a tie, use random choice to select one of them.
        """
        labels, counts = np.unique(result, return_counts=True)
        max_count = np.max(counts)
        max_indices = np.where(counts == max_count)[0]
        if len(max_indices) > 1:
            selected_index = np.random.choice(max_indices)
            return [labels[selected_index]]
        else:
            return [labels[np.argmax(counts)]]
    
    def predict_all(self, inp):
        
        """
            X is a matrix or 2-D numpy array, represnting testing instances. 
            Each testing instance is a feature vector. 
            
            Return the predictions of all instances in a list.
            
            TODO: 6. Revise the following for-loop to call ind_predict to get predictions
        """
        result = []
        for i in range(inp.shape[0]):
            result.append(self.ind_predict(inp[i]))
        return result

### Test Accruacy should be greater than 0.69

In [16]:
clf = RandomForest(n_trees= 100, n_split=None)
clf.fit(X_train, y_train)

In [17]:
pred = clf.predict_all(X_test)
(pred == y_test).mean()

IndexError: invalid index to scalar variable.